<a href="https://colab.research.google.com/github/federico-bassi/reddito_di_cittadinanza/blob/main/python/data_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labour Economics and Policy Evaluation Project: Reddito di Cittadinanza

The following notebook contains the code necessary to manipulate the data for the project and to build the dataset.

In [ ]:
import pandas as pd
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## choose the unit level among "regions" and "provinces"
granularity = "regions"

In [ ]:
path = f"../content/drive/MyDrive/LEPE_project/data/{granularity}/input_datasets"
files = glob.glob(os.path.join(path, "*.xlsx"))

In [ ]:
files

['../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/num_recipients.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/gdp.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/residents.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/perc_poor_families.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/perc_poor_individuals.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/unemployment.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/unemployment_no_degree.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/unemployment_25_34.xlsx',
 '../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/perc_applicants.xlsx']

In [ ]:
datasets = {}
for file in files:
  print(file)
  datasets[file.split(f"/data/{granularity}/input_datasets/")[1].split(".xlsx")[0]] = pd.read_excel(file, sheet_name="data", header=0)

../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/num_recipients.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/gdp.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/residents.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/perc_poor_families.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/perc_poor_individuals.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/unemployment.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/unemployment_no_degree.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/unemployment_25_34.xlsx
../content/drive/MyDrive/LEPE_project/data/regions/input_datasets/perc_applicants.xlsx


In [ ]:
unit_list = [u for u in list(datasets.values())[0]["unit"]]

In [ ]:
for key in datasets.keys():
  print("==================================")
  print("Evaluating dataset --> ", key)
  df = datasets[key]
  for unit in df["unit"]:
    if unit not in unit_list:
      print(unit)

Evaluating dataset -->  num_recipients
Evaluating dataset -->  gdp
Evaluating dataset -->  residents
Evaluating dataset -->  perc_poor_families
Evaluating dataset -->  perc_poor_individuals
Evaluating dataset -->  unemployment
Evaluating dataset -->  unemployment_no_degree
Evaluating dataset -->  unemployment_25_34
Evaluating dataset -->  perc_applicants


In [ ]:
def panelize(df, column_name):
  panel_df = df.melt(id_vars=["unit"], var_name="year", value_name= column_name)
  panel_df["year"] = panel_df["year"].astype(int)
  panel_df.set_index(["unit", "year"], inplace=True)
  return panel_df

In [ ]:
panelized_datasets = {}
for key in datasets.keys():
  panelized_datasets[key] = panelize(datasets[key], key)

In [ ]:
panelized_datasets = {key : value for key, value in sorted(panelized_datasets.items(), key= lambda x : len(x[1]), reverse=True)}

In [ ]:
dataset = pd.DataFrame()
counter = 1
for key in panelized_datasets.keys():
  df = panelized_datasets[key]
  print(key)
  if counter == 1:
    dataset = df
  else:
    dataset = dataset.merge(df, how="left", left_on = ["unit", "year"], right_on = ["unit", "year"])
  counter += 1

gdp
residents
perc_poor_families
perc_poor_individuals
unemployment
unemployment_no_degree
unemployment_25_34
num_recipients
perc_applicants


In [ ]:
dataset

,,gdp,residents,perc_poor_families,perc_poor_individuals,unemployment,unemployment_no_degree,unemployment_25_34,num_recipients,perc_applicants
unit,year,,,,,,,,,
Piemonte,2015,127078.3,4403562.0,6.6,8.9,10.3,15.002963,15.703800,NaN,NaN
Valle d'Aosta/Vallée d'Aoste,2015,4665.2,127972.0,7.2,10.5,8.9,18.390805,11.325942,NaN,NaN
Lombardia,2015,47409.3,9954769.0,8.5,12.5,9.2,13.350785,16.036811,NaN,NaN
Trentino Alto Adige/Südtirol,2015,364242.1,1056182.0,4.6,8.2,7.9,10.201484,9.376720,NaN,NaN
Veneto,2015,42009.4,4902694.0,..,5.2,5.3,5.628639,6.636754,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Puglia,2021,71533.8,3933777.0,27.5,32.2,14.6,18.000000,22.692705,346449.0,8.475324
Basilicata,2021,12282.0,545130.0,17.7,19.0,8.3,9.900000,14.117152,27075.0,0.701033
Calabria,2021,30040.5,1860601.0,20.3,24.1,18.0,21.000000,26.673685,240225.0,5.514995


In [ ]:
dataset.to_excel(f"../content/drive/MyDrive/LEPE_project/data/dataset_{granularity}.xlsx")